### Imports

In [ ]:
import numpy as np
import pandas as pd
import importlib
import os

import findatree.io as io
import findatree.transformations as transformations
import findatree.interactive as interactive
import findatree.geo_to_image as geo_to_image
import findatree.segmentation as segmentation
import findatree.photometric.shadow as shadow
import findatree.photometric.features as features

from bokeh.plotting import show, save
from bokeh.io import output_notebook, output_file

output_notebook()

### Definitions

In [ ]:
tnr = 11733

dir_names = []
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DSMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DTMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Orthos'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Kronen'])


dir_name_hdf5 = r"C:\Data\lwf\processed\2020\hdf5"
dir_name_html = r"C:\Data\lwf\analysis\px-wise_correlation\html"

### Load channels from .tif

In [ ]:
importlib.reload(geo_to_image)

params_channels = {
    'tnr': tnr,
    'px_width_reproject': 0.2,
    'downscale': 0,
}

# Load, reproject, normalize & downscale rasters
channels, params_channels = geo_to_image.channels_load(
    dir_names,
    params_channels,
)

# Save repojected rasters as channels in .hdf5
# io.channels_to_hdf5(channels, params_channels, dir_name = dir_name_hdf5)

### Load channels from .hdf5

In [ ]:
importlib.reload(io)

# Load data
data, params_data = io.load_hdf5(
    path=os.path.join(dir_name_hdf5, f"tnr{tnr}.hdf5"),
    groups=['channels'],
)

# Assign channels
channels = data['channels']
params_channels = params_data['channels']
del data, params_data

### Load human crowns

In [ ]:
importlib.reload(io)

# Load 'hand selected' human crowns
crowns_human, params_crowns_human = io.load_shapefile(
    dir_names,
    params_channels,
    remove_outliers = True,
    )

# Save crowns in .hdf5
# io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_name_hdf5)

### Add photometric features to human crowns

In [ ]:
importlib.reload(features)
importlib.reload(io)

# Add photometric features to human crowns
features.crowns_add_features(
    channels,
    params_channels,
    crowns_human,
    params_crowns_human,
)
# Save crowns in .hdf5
# io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_name_hdf5)

### View human crown features as DataFrame

In [ ]:
df = pd.merge(
    left = pd.DataFrame(crowns_human['features']['photometric']),
    right = pd.DataFrame(crowns_human['features']['terrestrial']),
    how= 'inner',
    on = ['id'],
    )

# df.loc[:,['ba','perc5_evi', 'perc25_evi','perc50_evi', 'perc75_evi','perc95_evi']].groupby(['ba']).apply('mean')

df.loc[:,['sst','perc5_evi', 'perc25_evi','perc50_evi', 'perc75_evi','perc95_evi']].groupby(['sst']).describe().loc[:, (slice(None), ['count', 'mean', 'std'])]




### Plotting

In [ ]:
importlib.reload(interactive)

plt = interactive.Plotter()
plt.width = 400
plt.channels_downscale = 0
plt.show_features = [
    'id',
    'ba', 'kkl','nbv',
    'perc25_ndvi','perc50_ndvi','perc75_ndvi',
]

plt.add_channels(channels, params_channels)

plt.figures_add_rgb(perc=2)
plt.figures_add_rgb(perc=2, color_code='533', weight_code = '133')
plt.figures_add_gray('evi')

plt.togglers_add_crowns(crowns_human, params_crowns_human)

layout = plt.create_layout()

In [ ]:
show(layout)

# output_file(os.path.join(dir_name_html, f"tnr{tnr}.html"), mode='inline')
# save(layout)